# Faster R-CNN 구현
### 텐서플로우 기반
### 주석은 한글 위주로 작성

In [1]:
import numpy as np
import os
import glob
import cv2
import xmltodict
import tensorflow as tf
import math
import random
from tqdm import tqdm
from PIL import Image, ImageDraw

## 훈련 이미지 가져오기

In [2]:
train_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/JPEGImages'
train_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/train/VOCdevkit/VOC2007/Annotations'

test_x_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/JPEGImages'
test_y_path = '/Users/minguinho/Documents/AI_Datasets/PASCAL_VOC_2007/test/VOCdevkit/VOC2007/Annotations'

In [3]:
list_train_x = sorted([x for x in glob.glob(train_x_path + '/**')])    
list_train_y = sorted([x for x in glob.glob(train_y_path + '/**')]) 

list_test_x = sorted([x for x in glob.glob(test_x_path + '/**')])    
list_test_y = sorted([x for x in glob.glob(test_y_path + '/**')]) 

print(len(list_train_x))
print(len(list_test_x))

5011
4952


훈련용 이미지는 5011개, 테스트용 이미지는 4952개가 있음을 알 수 있다. 그리고 둘다 x, y값이 존재한다. 

## 논문 흐름대로 코드 작성하려고 노력할 것

## 공용으로 사용하는 레이어 생성 
### ImageNet을 위해 훈련된 VGG16을 일부 가져온다

In [5]:
max_num = len(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers) # 레이어 최대 개수

SharedConvNet = tf.keras.models.Sequential()
for i in range(0, max_num-1):
    SharedConvNet.add(tf.keras.applications.VGG16(weights='imagenet', include_top=False,  input_shape=(224, 224, 3)).layers[i])

SharedConvNet.summary() # 13개의 레이어 공유(컨볼루션 레이어 10개, 풀링 레이어 3개). 논문에선 14*14라는데 아씨 모르겠다

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

## RPN - 앵커 준비
#### 입력 이미지 기준으로 앵커를 생성한다.
#### 풀링을 3번 하므로 2^3 = 8이니 (8, 8)부터 (16, 8)...등 8픽셀식 중심 좌표를 옮겨가며 앵커들을 k개씩 생성한다
#### 생성한 앵커들 중 사용할 가치가 있는 앵커를 걸러낸다(이미지 범위를 벗어나지 않는 앵커들만 선정)
#### 선정한 앵커 중 실제 물체의 box와 얼마나 곂치는지(IoU) 계산해본다. 확실히 겹친다 하는 애들을 Positive 앵커로, 거의 안겹친다 하는 애들은 Negataive 앵커로 선정한다. 애매한 애들은 거른다.
#### 이렇게 생성된 앵커들로 미니배치를 생성한다. Positive 128개, Negative 128개로 만드는게 ideal한 구성이긴 한데 Positive한 앵커가 별로 없다. 그래서 Positive를 128개 못채웠으면 Negataive 앵커로 채워준다. 

In [6]:
n = 3 # 논문에서 n은 VGG16을 거치고 나온 특성맵 위를 슬라이딩 할 커널의 크기

# 생성할 앵커 크기는? 
anchor_size = [32, 64, 128] # 이미지 크기가 224*224라 32, 64, 128로 지정
anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]] # W*L기준 


### 앵커를 만들어(224*224 기준) -> 여기서 positive 앵커, negative 앵커로 나눠 -> 얘들을 좀 비율 맞게 조합해서 256개의 앵커 그룹(미니배치)를 만들어 -> RPN훈련에 사용해

### 함수로 만드는 이유?
#### 이미지 별로 라벨에서 박스 좌표랑 객체 종류 뽑아내서 loss를 계산해야한다. 그래서 코드의 간결화?를 위해 함수를 만드는 것

In [7]:
# 앵커 생성 함수. 
def make_anchor(anchor_size, anchor_aspect_ratio, image_size = 224) :
    # 입력 이미지(그래봤자 224*224긴 하지만)에 맞춰 앵커를 생성해보자 

    anchors = [] # [x,y,w,h]로 이루어진 리스트 

    # 앵커 중심좌표 간격
    # 왜 간격이 8이냐? pooling을 3번 했기 때문에 각 특성맵의 픽셀 하나하나가 원래 이미지에서 8*8 픽셀을 대표하는 값들이다
    # 즉, 특성맵의 각 좌표 위를 슬라이딩 할건데 센터 좌표를 하나씩 건넌다는건 원래 이미지에서 8픽셀씩 움직인다는 것과 같다. 
    # 그렇기에 간격을 8*8로 설정한 것이다 
    interval_x = 16 
    interval_y = 16
    Center_max_x = 208 # 224 - 16, 중심좌표가 224가 될 수는 없다.
    Center_max_y = 208 # 224 - 16

    # 2단 while문 생성
    x = 16
    y = 16
    while(y <= 208):
        while(x <= 208):
            # k개의 앵커 생성. 여기서 k = len(anchor_size) * len(anchor_aspect_ratio)다
            for i in range(0, len(anchor_size)) : 
                for j in range(0, len(anchor_aspect_ratio)) :
                    anchor_width = anchor_aspect_ratio[j][0] * anchor_size[i]
                    anchor_height = anchor_aspect_ratio[j][1] * anchor_size[i]
                    # 얘를 사용할 수 있는 앵커인가? 필터링
                    if((x - (anchor_width/2) >= 0) and (y - (anchor_height/2) >= 0) and
                    (x + (anchor_width/2) <= 224) and (y + (anchor_height/2) <= 224)):
                        # 조건이 맞다고 판단되면 앵커 생성
                        anchor = [x, y, anchor_width, anchor_height]
                        anchors.append(anchor)
            x = x + interval_x 
        y = y + interval_y
        x = 16
    return anchors

In [8]:
anchors = make_anchor(anchor_size, anchor_aspect_ratio, 224)

len(anchors) # 4181개의 앵커 생성. 이 앵커 집합은 모든 이미지 입력에 적용할 앵커다

1117

#### 각 입력 이미지에서 객체의 Ground_Truth_Box, Class를 추출해야한다
#### Ground_Truth_Box는 RPN에서 IoU 구하는데 사용하기도 하고  Loss에서도 사용하기도 한다

#### Loss 사용을 위해 Class list도 만들자.
#### 논문의 loss 함수를 보면 pi가 있다. pi는 리스트인데 PASCAL VOC에 존재하는 객체들이 몇 %의 확률로 있느냐 나타내는거다
#### 이를 위해 어떤 객체들이 PASCAL VOC에 존재하는지 알아야한다. 

In [9]:
# label 추출을 위한 label 파일 리스트. label 정보가 xml파일 안에 있다.
xml_file_list = sorted([x for x in glob.glob(train_y_path + '/**')])

In [10]:
# 존재하는 객체 종류를 알아내자
def get_Classes_inImage():
    classes = []

    for xml_file_path in xml_file_list: 

        f = open(xml_file_path)
        xml_file = xmltodict.parse(f.read())
        # 사진에 객체가 여러개 있을 경우
        try: 
            for obj in xml_file['annotation']['object']:
                classes.append(obj['name'].lower()) # 들어있는 객체 종류를 알아낸다
        # 사진에 객체가 하나만 있을 경우
        except TypeError as e: 
            classes.append(xml_file['annotation']['object']['name'].lower()) 
        f.close()

    classes = list(set(classes)) # set은 중복된걸 다 제거하고 유니크한? 아무튼 하나만 가져온다. 그걸 리스트로 만든다
    classes.sort() # 정렬

    return classes

In [11]:
label_classes = get_Classes_inImage()

In [12]:
# IoU를 위한 Ground_Truth_Box와 Loss를 위한 Class 리스트를 얻는다. 
def get_label_fromImage(xml_file_path):

    f = open(xml_file_path)
    xml_file = xmltodict.parse(f.read()) 

    # 우선 원래 이미지 크기를 얻는다. 왜냐하면 앵커는 224*224 기준으로 만들었는데 원본 이미지는 224*224가 아니기 때문.
    # 224*224에 맞게 줄일려고 하는거다
    Image_Width = float(xml_file['annotation']['size']['height'])
    Image_Height  = float(xml_file['annotation']['size']['width'])


    Classes_list = [] 
    Ground_Truth_Box_list = [] 

    # multi-objects in image
    try:
        for obj in xml_file['annotation']['object']:
            obj_class = obj['name'].lower() 
            # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
            x_min = float(obj['bndbox']['xmin']) 
            y_min = float(obj['bndbox']['ymin'])
            x_max = float(obj['bndbox']['xmax']) 
            y_max = float(obj['bndbox']['ymax'])

            # 224*224에 맞게 변형시켜줌
            x_min = float((224/Image_Width)*x_min)
            y_min = float((224/Image_Height)*y_min)
            x_max = float((224/Image_Width)*x_max)
            y_max = float((224/Image_Height)*y_max)


            # 얘들을 앵커랑 같은 양식으로 저장(x,y,w,h)
            Center_x = (x_min + x_max)/2
            Center_y = (y_min + y_max)/2
            Ground_Truth_Box_Width = x_max - x_min
            Ground_Truth_Box_Height = y_max - y_min

            Ground_Truth_Box = [Center_x, Center_y, Ground_Truth_Box_Width, Ground_Truth_Box_Height] 

            index = label_classes.index(obj_class) 

            Classes_list.append(index)
            Ground_Truth_Box_list.append(Ground_Truth_Box)

    # single-object in image
    except TypeError as e : 
        
        obj_class = xml_file['annotation']['object']['name'] 
        # 박스 좌표(왼쪽 위, 오른쪽 아래) 얻기
        x_min = float(xml_file['annotation']['object']['bndbox']['xmin']) 
        y_min = float(xml_file['annotation']['object']['bndbox']['ymin']) 
        x_max = float(xml_file['annotation']['object']['bndbox']['xmax']) 
        y_max = float(xml_file['annotation']['object']['bndbox']['ymax']) 

        # 224*224에 맞게 변형시켜줌
        x_min = float((224/Image_Width)*x_min)
        y_min = float((224/Image_Height)*y_min)
        x_max = float((224/Image_Width)*x_max)
        y_max = float((224/Image_Height)*y_max)


        # 얘들을 앵커랑 같은 양식으로 저장(x,y,w,h)
        Center_x = (x_min + x_max)/2
        Center_y = (y_min + y_max)/2
        Ground_Truth_Box_Width = x_max - x_min
        Ground_Truth_Box_Height = y_max - y_min

        Ground_Truth_Box = [Center_x, Center_y, Ground_Truth_Box_Width, Ground_Truth_Box_Height] 

        index = label_classes.index(obj_class) 

        Classes_list.append(index)
        Ground_Truth_Box_list.append(Ground_Truth_Box)


    return Classes_list, Ground_Truth_Box_list


In [12]:
# 테스트. xml_file_list[1]에 해당하는 이미지를 갖고 학습한다고 가정
Class_label, Ground_Truth_Box_list = get_label_fromImage(xml_file_list[0])
print(Class_label) # car가 이미지에 있고
print(Ground_Truth_Box_list) # 224*224로 변환했을 때 약 (215, 85) 좌표에 (241, 125) 크기의 Ground_Truth_Box가 있음을 알 수 있다. 

[8, 8, 8, 8, 8]
[[175.31733333333335, 123.20000000000002, 36.437333333333356, 57.34400000000001], [124.84266666666667, 142.464, 52.56533333333334, 48.384], [21.504, 138.432, 37.03466666666667, 58.239999999999995], [160.08533333333332, 110.432, 32.256, 47.03999999999999], [175.91466666666668, 90.944, 20.906666666666666, 15.232]]


In [20]:
# 앵커들을 Positive, Negative 앵커로 나누자
def align_anchor(xml_file_path):
    Positive_Anchor = []
    Negative_Anchor = []

    Claases_RelativeToPositiveAnchor = [] # Positive Anchor와 연관있는 클래스의 인덱스
    GroundTruthBox_RelativeToPositiveAnchor = [] # Positive Anchor와 연관있는 클래스의 박스

    # 미니배치에서 Negative Anchor와 연관있는 클래스의 인덱스, 박스가 필요한 경우가 있을 수 있다. 그러니 저장한다
    Claases_RelativeToNegativeAnchor = []
    GroundTruthBox_RelativeToNegativeAnchor = []

    Classes_list, Ground_Truth_Box_list = get_label_fromImage(xml_file_path) 

    for i in range(0, len(anchors)): # 모든 앵커에 대한 IoU 계산
        # 각 앵커에 대해 존재하는 모든 Ground_Truth_Box의 IoU를 계산한다.
        # 어떤 객체든 IoU가 0.7이 넘어가면 바로 Positive 를 붙혀준다.
        IoU_list = [] 
        
        for j in range(0, len(Ground_Truth_Box_list)):
            # IoU 계산
            # 왼쪽 x좌표 기준으로는 ground_truth와 anchor 중 max를, 오른쪽 기준으로는 min값을 사용한다.
            # 그림을 그려서 왼쪽 좌표끼리 묶어서 max, 오른쪽 좌표끼리 묶어서 min을 선택해보면 이해가 쉬울거다.
            Ground_Truth_Box_min_x = Ground_Truth_Box_list[j][0] - (Ground_Truth_Box_list[j][2]/2)
            Ground_Truth_Box_min_y = Ground_Truth_Box_list[j][1] - (Ground_Truth_Box_list[j][3]/2)
            Ground_Truth_Box_max_x = Ground_Truth_Box_list[j][0] + (Ground_Truth_Box_list[j][2]/2)
            Ground_Truth_Box_max_y = Ground_Truth_Box_list[j][1] + (Ground_Truth_Box_list[j][3]/2)
            
            anchor_min_x = anchors[i][0] - (anchors[i][2]/2)
            anchor_min_y = anchors[i][1] - (anchors[i][3]/2)
            anchor_max_x = anchors[i][0] + (anchors[i][2]/2)
            anchor_max_y = anchors[i][1] + (anchors[i][3]/2)

            IoU_minX = max(anchor_min_x, Ground_Truth_Box_min_x)
            IoU_minY = max(anchor_max_x, Ground_Truth_Box_max_x)
            IoU_maxX = min(anchor_min_x, Ground_Truth_Box_min_x)
            IoU_maxY = min(anchor_min_x, Ground_Truth_Box_min_x)

            Intersection_Area = ((IoU_maxX - IoU_minX) * (IoU_maxY - IoU_minY)) # 교집합 넓이
            Union_Area = (anchors[i][2] * anchors[i][3]) + (Ground_Truth_Box_list[j][2] * Ground_Truth_Box_list[j][3]) - Intersection_Area # 합집합 넓이?

            IoU = Intersection_Area/Union_Area # IoU를 구했다

            IoU_list.append(IoU)

        # Ground_Truth_Box별로 IoU를 얻었다. 이제 Positive, Negative Anchor를 구분하자.
        # Loss에서 박스 위치관련 로스 계산할 때 어떤 객체의 박스를 기준으로 positive 앵커가 되었는지 물어본다. 그래서 그것도 저장하려고 한다.  
        for j in range(0, len(IoU_list)):
            # Positive는 기준이 2개 있긴하지만 난 2번 기준(IoU가 0.7 이상)만 쓰겠다. any groud_truth_box에 대한 IoU가 0.7보다 크면 된다. 
            if IoU_list[j] > 0.7 : 
                Positive_Anchor.append(anchors[i])
                Classes_RelativeToPositiveAnchor.append(Classes_list[j])
                GroundTruthBox_RelativeToPositiveAnchor(Ground_Truth_Box_list[j])
                break
        # Negative는 IoU가 0.3보다 작은 앵커를 사용한다. 모든 Ground_Truth_Box에 대한 IoU가 0.3보다 작은 앵커가 negative 앵커다.
        if max(IoU_list) < 0.3 : 
            index = IoU_list.index(max(IoU_list))
            Negative_Anchor.append(anchors[i])
            Classes_RelativeToNegativeAnchor.append(Classes_list[index])
            GroundTruthBox_RelativeToNegativeAnchor.append(Ground_Truth_Box_list[index])


    
    return Positive_Anchor, Negative_Anchor, Classes_RelativeToPositiveAnchor, GroundTruthBox_RelativeToPositiveAnchor, Classes_RelativeToNegativeAnchor, GroundTruthBox_RelativeToNegativeAnchor


### 훈련을 위한 미니배치 256(positive 128, negative 128개) 선발

In [21]:
def Create_Minibatch(Positive_Anchor, Negative_Anchor, Classes_RelativeToPositiveAnchor, GroundTruthBox_RelativeToPositiveAnchor, Classes_RelativeToNegativeAnchor, GroundTruthBox_RelativeToNegativeAnchor) : 

    max_for = min([120, len(Positive_Anchor)])

    ran_list = random.sample(range(0, max_for), 120)
    Anchor_Minibatch = []
    Claases_RelativeToPositiveAnchor_forMinibatch = [] # reg Loss에 쓰임
    GroundTruthBox_RelativeToPositiveAnchor_forMinibatch = [] # reg Loss에 쓰임
    for random_num in ran_list :
        Anchor_Minibatch.append(Positive_Anchor[random_num])
        Claases_RelativeToPositiveAnchor_forMinibatch.append(Claases_RelativeToPositiveAnchor[random_num])
        GroundTruthBox_RelativeToPositiveAnchor_forMinibatch(GroundTruthBox_RelativeToPositiveAnchor[random_num])

    # positive Anchor가 120개 미만일 경우 negative anchor에서 빈걸 채워줌
    ran_list = random.sample(range(0, len(Negative_Anchor)), 120 - len(GroundTruthBox_RelativeToPositiveAnchor_forMinibatch))

    if len(GroundTruthBox_RelativeToPositiveAnchor_forMinibatch) < 120 :
        for random_num in ran_list :
            Anchor_Minibatch.append(Negative_Anchor[random_num])
            Claases_RelativeToPositiveAnchor_forMinibatch.append(Classes_RelativeToNegativeAnchor[random_num])
            GroundTruthBox_RelativeToPositiveAnchor_forMinibatch(GroundTruthBox_RelativeToNegativeAnchor[random_num])

    # 이제 negative 앵커
    ran_list = random.sample(range(0, len(Negative_Anchor)), 120) # 랜덤성 증가?를 위해 또다시 난수 생성
    for random_num in ran_list :
        Anchor_Minibatch.append(Negative_Anchor[random_num])
    # 0~127까지 Positive, 128~255까지 Negative

    return Anchor_Minibatch, Claases_RelativeToPositiveAnchor_forMinibatch, GroundTruthBox_RelativeToPositiveAnchor_forMinibatch

### Loss 함수 생성

In [26]:
def get_ouput_forLoss(cls_layer_output, reg_layer_output, Anchor_Minibatch):
    # cls_layer_output는 18개의 채널을 가진 14*14 특성맵이다. (14*14*18)
    # reg_layer_output는 36개의 채널을 가진 14*14 특성맵이다. (14*14*36)

    # 출력값 중 loss함수를 위해 선별되는 것들
    reg_layer_output_forMinibatch = []
    cls_layer_output_forMinibatch = []

    for Anchor in Anchor_Minibatch :
        # '특성맵에서' 앵커의 중심좌표
        anchor_x_inFeatureMap = (Anchor[0]/16) - 1
        anchor_y_inFeatureMap = (Anchor[1]/16) - 1

        anchor_index = 0 # 9개의 앵커가 있다고 헸다

        # anchor_size = [32, 64, 128]
        # anchor_aspect_ratio = [[1,1],[1,0.5], [0.5,1]]
        # 앵커 생성할 때 [32,32], [32,16], [16, 32], [64, 64]...순서로 만들었기에 여기선 [32,32] 앵커의 인덱스를 0이라 설정한다. 

        aspect_ratio = Anchor[2]/Anchor[3]
        if aspect_ratio == 1 : 
            if Anchor[2] == 32 :
                anchor_index = 0
            elif Anchor[2] == 64 :
                anchor_index = 3
            elif Anchor[2] == 128 :
                anchor_index = 6

        elif aspect_ratio == 0.5 : 
            if Anchor[2] == 32 :
                anchor_index = 1
            elif Anchor[2] == 64 :
                anchor_index = 4
            elif Anchor[2] == 128 :
                anchor_index = 7

        elif aspect_ratio == 2 : 
            if Anchor[2] == 32 :
                anchor_index = 2
            elif Anchor[2] == 64 :
                anchor_index = 5
            elif Anchor[2] == 128 :
                anchor_index = 8
        
        cls_output_forThisAnchor = [cls_layer_output[anchor_x_inFeatureMap][anchor_y_inFeatureMap][anchor_index], cls_layer_output[anchor_x_inFeatureMap][anchor_y_inFeatureMap][anchor_index + 1]]
        reg_output_forThisAnchor = [reg_layer_output[anchor_x_inFeatureMap][anchor_y_inFeatureMap][anchor_index], cls_layer_output[anchor_x_inFeatureMap][anchor_y_inFeatureMap][anchor_index + 1], reg_layer_output[anchor_x_inFeatureMap][anchor_y_inFeatureMap][anchor_index + 2], cls_layer_output[anchor_x_inFeatureMap][anchor_y_inFeatureMap][anchor_index + 3]] # (x, y, w, h로 지정)

        cls_layer_output_forMinibatch.append(cls_output_forThisAnchor)
        reg_layer_output_forMinibatch.append(reg_output_forThisAnchor)
        
    return cls_layer_output_forMinibatch, reg_layer_output_forMinibatch



In [27]:
def Smooth_L1(ti, ti_star) :
    difference_ti = ti - ti_star
    smooth_L1 = 0
    if abs(difference_ti) < 1: smooth_L1 = 0.5 * (difference_ti * difference_ti)
    else : smooth_L1 = abs(difference_ti) - 0.5

    return smooth_L1

In [28]:
# Loss_Regression. Lreg에 해당
def Loss_Regression(predict_box, anchor_box, groundTruth_box) : 
    # 와씨..이게 그 가스라이팅? 그거냐?
    # predict_box는 36개의 앵커박스 구성요소, 한 픽셀에 대한 36개의 앵커박스의 x,y,w,h가 쫘르륵...있는건데 거기서 x,y,w,h 한웅큼 가져온거다.
    # x,y,w,h로 가져오는건 뭐 알아서 하겠지...미래의 내가
    t_x = (predict_box[0] - anchor_box[0])/anchor_box[2]
    t_y = (predict_box[1] - anchor_box[1])/anchor_box[3]
    t_w = math.log10(predict_box[2]/anchor_box[2])
    t_h = math.log10(predict_box[3]/anchor_box[3])

    t_x_star = (groundTruth_box[0] - anchor_box[0])/anchor_box[2]
    t_y_star = (groundTruth_box[1] - anchor_box[1])/anchor_box[3]
    t_w_star = math.log10(groundTruth_box[2]/anchor_box[2])
    t_h_star = math.log10(groundTruth_box[3]/anchor_box[3])

    # Smooth L1 구하기
    # 구성요소가 4개니까 4번 구해야겠지?
    smooth_L1_x = Smooth_L1(t_x, t_x_star)
    smooth_L1_y = Smooth_L1(t_y, t_y_star)
    smooth_L1_w = Smooth_L1(t_w, t_w_star)
    smooth_L1_h = Smooth_L1(t_h, t_h_star)

    smooth_L1_list = [smooth_L1_x, smooth_L1_y, smooth_L1_w, smooth_L1_h]

    return smooth_L1_list # 모아서 반환


In [29]:
# L_cls
def Loss_Classes(cls_layer_output, pi_ground_truth) :
    log_loss = -pi_ground_truth*math.log10(cls_layer_output[0])-(1-pi_ground_truth)*math.log10(1-cls_layer_output)
    return log_loss

In [48]:
# Loss함수 수행. 입력 이미지의 라벨값을 받는다.
# loss는 아웃풋을 뽑아낸 뒤에 얻어야한다. 
def Loss_RPN(cls_layer_output, reg_layer_output, xml_file_path): # RPN에서 최종 결과값인 reg_layer와 cls layer의 output, 그리고 라벨이 들어있는 파일

    Positive_Anchor, Negative_Anchor, Claases_RelativeToPositiveAnchor, GroundTruthBox_RelativeToPositiveAnchor = align_anchor(xml_file_path) # 앵커 분류

    Anchor_Minibatch, Claases_RelativeToPositiveAnchor_forMinibatch, GroundTruthBox_RelativeToPositiveAnchor_forMinibatch = Create_Minibatch(Positive_Anchor, Negative_Anchor, Claases_RelativeToPositiveAnchor, GroundTruthBox_RelativeToPositiveAnchor) # 미니배치

    

    # 자, 여기서 잠깐 생각을 해보자
    # k = 9로 했으니 cls_layer의 결과값은 18개, reg_layer의 결과값은 36개가 있다. 
    # 앵커를 256개 랜덤해서 뽑겠다는 말이니까 결과값도 256쌍을 뽑아야한다. 그렇지? 이 256쌍은 뽑힌 앵커에 해당하는 출력값을 뽑아야한다. 
    cls_layer_output_forMinibatch, reg_layer_output_forMinibatch = get_ouput_forLoss(cls_layer_output, reg_layer_output, Anchor_Minibatch) # 선발된 앵커에 맞춰 output도 걸러낸다

    N_cls = len(Anchor_Minibatch)
    N_reg = 14*14 # VGG 특성맵 최종 output 넓이. 근데 이거 바꿔야한다. 28*28이 아니라 14*14더라
    lambda_forLoss = 10

    Lcls_sum = 0
    Lreg_sum = 0

    # RPN의 cls output이 pi, reg  output이 ti이다.
    # k = 9일 때 pi는 18개, ti는 36개라는 말.
    for i in range(0, len(Anchor_Minibatch)) :
        pi_ground_truth = 0
        if i < 128 : # Positive Anchor
            pi_ground_truth = 1
        #Lcls(classes loss)
        Lcls_sum = Lcls_sum + Loss_Classes(cls_layer_output_forMinibatch[i], pi_ground_truth)

        #Lreg(regression loss)
        if i < 128 : 
            Lreg_sum = Lreg_sum + pi_ground_truth * Loss_Regression(reg_layer_output_forMinibatch, Anchor_Minibatch[i], GroundTruthBox_RelativeToPositiveAnchor_forMinibatch[i])

    loss = (1/N_cls) * Lcls_sum + lambda_forLoss*(1/N_reg)*Lreg_sum

    return loss

### RPN 생성

In [45]:
k = len(anchor_size) * len(anchor_aspect_ratio) # 3*3 = 9

initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)

RPN_intermediate_layer = tf.keras.layers.Conv2D(512, (n,n), activation='relu', padding='VALID', kernel_initializer = initializer, input_shape=(14, 14, 512))
RPN_cls_Layer = tf.keras.layers.Conv2D(2*k, (1,1), activation='sigmoid', padding='VALID', kernel_initializer = initializer,input_shape=(14, 14, 512)) # 클래스인가? 아닌가? 
RPN_reg_Layer = tf.keras.layers.Conv2D(4*k, (1,1), activation='relu', padding='VALID', kernel_initializer = initializer,input_shape=(14, 14, 512))

In [49]:
image_file_list = sorted([x for x in glob.glob(train_x_path + '/**')])
xml_file_list = sorted([x for x in glob.glob(train_y_path + '/**')]) # 있긴한데 또 만들었다. 통일성을 위해.

## 훈련을 위한 함수

In [61]:
# 입력용 이미지 생성. 224, 224로 변환시켜준다. 
def make_input():
    
    images_list = []
    
    for i in tqdm(range(0, len(image_file_list))) :
        
        image = cv2.imread(image_file_list[i])
        image = cv2.resize(image, (224, 224))/255
        
        images_list.append(image)
    
    return np.asarray(images_list)

In [88]:
# 공유해서 사용할 특성맵 리스트들을 출력하자
def make_Shared_ConvLayer_FeatureMap(SharedConvNet) :
    image_list = make_input()

    shared_Conv_output_list = np.array([])
    for i in tqdm(range(0, len(image_list))): 
        shared_Conv_output = SharedConvNet(np.expand_dims(image_list[i], axis=0))
        shared_Conv_output_list = np.append(shared_Conv_output_list, shared_Conv_output)

    return shared_Conv_output_list

In [89]:
def training_RPN(shared_Conv_output_list, RPN_intermediate_layer, RPN_cls_Layer, RPN_reg_Layer) :

    for i in tqdm(Range(0, len(image_file_list))) :
        input_label_path = xml_file_list[i]
    
        intermedi_output = RPN_intermediate_layer(np.expand_dims(shared_Conv_output_list[i], axis=0)) # 출력 = 레이어(입력)
        cls_layer_output = RPN_cls_Layer(intermedi_output)
        reg_layer_output = RPN_reg_Layer(intermedi_output)

        # 각 이미지에서 미니배치를 선발 -> 손실 구하기(한 입력값에 대한 손실)
        loss = Loss_RPN(cls_layer_output, reg_layer_output, xml_file_list[i])

        # 학습률 0.001 for 60k mini batch, 학습률 0.0001 for next 20k mini-batches
        # momentum : 0.9, weight decay : 0.0005


    return RPN_intermediate_layer, RPN_cls_Layer, RPN_reg_Layer

In [ ]:
# 여러가지 앵커를 ground_truth_box와 IoU 비교후 앵커들을 선정 -> RoI로 사용.
# RoI에 해당하는 영역을 특성맵에서 찾아야하는데 RoI는 224*224 기준이고 특성맵은 14*14임. -> 특성맵에 틀어맞게 RoI를 변환 -> 변환했으니 14*14 내부에 RoI가 변환되어 있음 -> 변환되어있는 RoI들을 FCs 2개에 넣어줌 -> 'class 레이어'와 'box 위치 분류 레이어' 두 곳에 넣어줌
# Roi Pooling만 잘 구현하면 되겠는데...어찌 구현할까(21/4/20)

In [90]:
shared_Conv_output_list = make_Shared_ConvLayer_FeatureMap(SharedConvNet)

cls_layer_output_list = np.array([])
reg_layer_output_list = np.array([])

# 결과값이 어떻게 나올까? 궁금하다
for i in tqdm(range(0, len(image_file_list))) :
    intermedi_output = RPN_intermediate_layer(np.expand_dims(shared_Conv_output_list[i], axis=0)) # 출력 = 레이어(입력)
    cls_layer_output = RPN_cls_Layer(intermedi_output)
    reg_layer_output = RPN_reg_Layer(intermedi_output)

    cls_layer_output_list = np.append(cls_layer_output_list, cls_layer_output)
    reg_layer_output_list = np.append(reg_layer_output_list, reg_layer_output)

100%|██████████| 5011/5011 [2:03:40<00:00,  1.48s/it]


NameError: name 'Range' is not defined

Fast RCNN의 로스(따로 만들어야한다. 위에 Loss는 RPN의 Loss다.), 모델 구조를 만들고 4-step 교차 훈련방법을 사용하자
<br>
그리고 로스를 이용해 가중치를 업데이트하는 방법(수동으로)을 알아야한다. 이걸 모르면 진행이 안된다. 
<br>
근데 그 방법을 알면 바로 만들 수 있다. 